In [1]:
"""Test script that runs all the vehicles in both Excel and Python FASTSim for both UDDS and HWFET cycles."""

import xlwings as xw
import pandas as pd
import time
import numpy as np
import os
import sys
import importlib
import xlwings as xw
from math import isclose
import importlib
import pickle

# local modules
from fastsim.tests import test_vs_excel

In [2]:
res_python = test_vs_excel.run_python(verbose=False, use_jit=False)

Instantiating classes.

running PHEV SOC iteration
running PHEV SOC iteration
running PHEV SOC iteration
running PHEV SOC iteration
running PHEV SOC iteration
running PHEV SOC iteration
running PHEV SOC iteration
running PHEV SOC iteration
running PHEV SOC iteration
running PHEV SOC iteration

Elapsed time:  21.61 s


In [3]:
res_excel = test_vs_excel.run_excel('res_excel.p')

In [4]:
res_comps = test_vs_excel.compare(res_python, res_excel)



2016 MITSUBISHI i-MiEV
*********************
All values within error tolerance of 0.001


2016 TOYOTA Corolla 4cyl 2WD
*********************
All values within error tolerance of 0.001


2016 KIA Optima Hybrid
*********************
All values within error tolerance of 0.001


2016 CHEVROLET Malibu 4cyl 2WD
*********************
All values within error tolerance of 0.001


2016 HYUNDAI Elantra 4cyl 2WD
*********************
All values within error tolerance of 0.001


2016 Hyundai Tucson Fuel Cell
*********************
All values within error tolerance of 0.001


2016 BMW i3 REx PHEV
*********************
You ran a PHEV, which is not working yet in python


2016 CHEVROLET Spark EV
*********************
All values within error tolerance of 0.001


2016 AUDI A3 4cyl 2WD
*********************
All values within error tolerance of 0.001


2016 FORD C-MAX (PHEV)
*********************
You ran a PHEV, which is not working yet in python


2016 CHEVROLET Volt
*********************
You ran a PHEV

In [23]:
vehstr = "2016 AUDI A3 4cyl 2WD"
for key in res_excel[vehstr].keys():
    print(key + ' : {', end='')
    print('excel : {:.4g}, '.format(res_excel[vehstr][key]), end='')
    print('python:{:.4g}'.format(res_python[vehstr][key]) + '}')

labUddsMpgge : {excel : 39.98, python:39.98}
labHwyMpgge : {excel : 49.96, python:49.96}
labCombMpgge : {excel : 43.93, python:43.93}
labUddsKwhPerMile : {excel : 0, python:0}
labHwyKwhPerMile : {excel : 0, python:0}
labCombKwhPerMile : {excel : 0, python:0}
adjUddsMpgge : {excel : 30.5, python:30.5}
adjHwyMpgge : {excel : 35.3, python:35.3}
adjCombMpgge : {excel : 32.49, python:32.49}
adjUddsKwhPerMile : {excel : 0, python:0}
adjHwyKwhPerMile : {excel : 0, python:0}
adjCombKwhPerMile : {excel : 0, python:0}


In [25]:
vehstr = '2016 FORD C-MAX HEV'
for key in res_excel[vehstr].keys():
    print(key + ' : {', end='')
    print('excel : {:.4g}, '.format(res_excel[vehstr][key]), end='')
    print('python:{:.4g}'.format(res_python[vehstr][key]) + '}')

labUddsMpgge : {excel : 57.8, python:57.8}
labHwyMpgge : {excel : 54.68, python:54.68}
labCombMpgge : {excel : 56.35, python:56.35}
labUddsKwhPerMile : {excel : 0, python:0}
labHwyKwhPerMile : {excel : 0, python:0}
labCombKwhPerMile : {excel : 0, python:0}
adjUddsMpgge : {excel : 42.22, python:42.22}
adjHwyMpgge : {excel : 38.46, python:38.46}
adjCombMpgge : {excel : 40.44, python:40.44}
adjUddsKwhPerMile : {excel : 0, python:0}
adjHwyKwhPerMile : {excel : 0, python:0}
adjCombKwhPerMile : {excel : 0, python:0}
